In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
import pytz
from src.inference import (
    get_feature_store,
    fetch_next_hour_predictions,
    fetch_predictions,
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    est = pytz.timezone('US/Eastern')
    current_est_time = current.astimezone(est)
    return current_est_time

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current US/EST hour (ceiled): {current_hour}")

Current US/EST hour (ceiled): 2025-05-09 02:00:00-04:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='US/Eastern').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-09 01:30:09,991 INFO: Initializing external client
2025-05-09 01:30:09,991 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:30:11,995 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 


In [7]:
df

,start_station_id,predicted_demand,start_hour
0,6230.02,5.0,2025-05-09 06:00:00+00:00
1,5872.10,5.0,2025-05-09 06:00:00+00:00
2,5308.04,9.0,2025-05-09 06:00:00+00:00


In [8]:
df["start_hour"] = df["start_hour"].dt.tz_convert('US/Eastern')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                     
---  ------            --------------  -----                     
 0   start_station_id  3 non-null      float64                   
 1   predicted_demand  3 non-null      float64                   
 2   start_hour        3 non-null      datetime64[us, US/Eastern]
dtypes: datetime64[us, US/Eastern](1), float64(2)
memory usage: 204.0 bytes


In [9]:
predictions = fetch_next_hour_predictions()

2025-05-09 01:31:01,105 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:31:01,133 INFO: Initializing external client
2025-05-09 01:31:01,133 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:31:02,487 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 
Current UTC time: 2025-05-09 05:31:01.089932+00:00
Next hour UTC: 2025-05-09 06:00:00+00:00; Next Hour Eastern: 2025-05-09 02:00:00-04:00
Found 3 records


In [10]:
predictions

,start_station_id,predicted_demand,start_hour
0,6230.02,5.0,2025-05-09 02:00:00-04:00
1,5872.10,5.0,2025-05-09 02:00:00-04:00
2,5308.04,9.0,2025-05-09 02:00:00-04:00


In [11]:
import pandas as pd
current_date = pd.Timestamp.now(tz='US/Eastern').ceil('h')

In [12]:
dt = current_date.ceil('h')
dt

Timestamp('2025-05-09 02:00:00-0400', tz='US/Eastern')

In [13]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all()
query = query.filter(fg.start_hour > "2025-03-03 05:00:00")  
results = query.read()

2025-05-09 01:32:00,703 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:32:00,730 INFO: Initializing external client
2025-05-09 01:32:00,730 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:32:01,988 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 


In [14]:
fg.filter((fg.start_hour >= "2025-03-03") & (fg.start_hour < "2025-03-04")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 


,start_station_id,predicted_demand,start_hour


In [15]:
results["start_hour"] = results["start_hour"].dt.tz_convert('US/Eastern')

results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype                     
---  ------            --------------  -----                     
 0   start_station_id  3 non-null      float64                   
 1   predicted_demand  3 non-null      float64                   
 2   start_hour        3 non-null      datetime64[us, US/Eastern]
dtypes: datetime64[us, US/Eastern](1), float64(2)
memory usage: 204.0 bytes


In [16]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["start_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [17]:
predictions = fetch_next_hour_predictions()

2025-05-09 01:34:45,372 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:34:45,384 INFO: Initializing external client
2025-05-09 01:34:45,384 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:34:46,883 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Current UTC time: 2025-05-09 05:34:45.372583+00:00
Next hour: 2025-05-09 06:00:00+00:00
Found 3 records


In [18]:
predictions

,start_station_id,predicted_demand,start_hour
0,6230.02,5.0,2025-05-09 02:00:00-04:00
1,5872.10,5.0,2025-05-09 02:00:00-04:00
2,5308.04,9.0,2025-05-09 02:00:00-04:00


In [19]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-05-09 01:35:10,859 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:35:10,872 INFO: Initializing external client
2025-05-09 01:35:10,874 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:35:12,255 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682


In [20]:
df = fg.read()
df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.48s) 


,start_station_id,predicted_demand,start_hour
0,6230.02,5.0,2025-05-09 06:00:00+00:00
1,5872.10,5.0,2025-05-09 06:00:00+00:00
2,5308.04,9.0,2025-05-09 06:00:00+00:00


In [21]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-09 06:00:00+00:00


In [22]:
df[df["start_hour"] == next_hour]

,start_station_id,predicted_demand,start_hour
0,6230.02,5.0,2025-05-09 06:00:00+00:00
1,5872.10,5.0,2025-05-09 06:00:00+00:00
2,5308.04,9.0,2025-05-09 06:00:00+00:00
